In [3]:
#Code to help get all csv files into a useable format for making into figures
import pandas as pd
import numpy as np
import sys
import os
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
outDir = thisDir + 'output/mLmax/results/ridge/'
subDir=outDir+'subNetwork/'
ranDir=outDir+'randomIdx/'

taskList=['mixed','mem','motor']
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
#all permutations of features
rand_CV=pd.DataFrame()
rand_SS=pd.DataFrame()
rand_DS=pd.DataFrame()
rand_BS=pd.DataFrame()
features=[10,20,30,40,50,60,70,80,90,100,200,300,400,500,1000,1500,2000,2500,3000,4000,5000,10000,15000,20000,30000,40000,50000]
for feat in features:
    #CV
    CVfile=pd.read_csv(ranDir+'CV/'+str(feat)+'.csv')
    CVfile.drop(columns='sub', inplace=True)
    CVfile=CVfile.melt(value_vars=['mixed', 'motor','mem'], var_name='Task', value_name='Accuracy')
    CVfile['Features']=feat
    rand_CV=pd.concat([rand_CV,CVfile])
    #SS
    SSfile=pd.read_csv(ranDir+'SS/'+str(feat)+'.csv')
    SSfile['Features']=feat
    rand_SS=pd.concat([rand_SS,SSfile])
    #DS
    DSfile=pd.read_csv(ranDir+'DS/'+str(feat)+'.csv')
    DSfile['Features']=feat
    rand_DS=pd.concat([rand_DS,DSfile])
    #BS
    BSfile=pd.read_csv(ranDir+'BS/'+str(feat)+'.csv')
    BSfile['Features']=feat
    rand_BS=pd.concat([rand_BS,BSfile])
    
rand_CV.to_csv(ranDir+'CV/all.csv')
rand_SS.to_csv(ranDir+'SS/all.csv')
rand_DS.to_csv(ranDir+'DS/all.csv')
rand_BS.to_csv(ranDir+'BS/all.csv')


#allFolds
#DS
motor=pd.DataFrame()
mixed=pd.DataFrame()
mem=pd.DataFrame()
for train in subList:
    for test in subList:
        if train==test:
            continue
        #motor
        tmpmotor=pd.read_csv(outDir+'acc/DS/folds/'+train+test+'motor.csv',usecols=[1,2,3])
        motor=pd.concat([motor,tmpmotor])
        #mixed
        tmpmix=pd.read_csv(outDir+'acc/DS/folds/'+train+test+'mixed.csv',usecols=[1,2,3])
        mixed=pd.concat([mixed,tmpmix])
        #mem
        tmpmem=pd.read_csv(outDir+'acc/DS/folds/'+train+test+'mem.csv',usecols=[1,2,3])
        mem=pd.concat([mem,tmpmem])
        
motMix=pd.merge(motor,mixed,on=['train', 'test'])        
DS_folds=pd.merge(motMix,mem,on=['train', 'test'])   
DS_folds.rename(columns={'mixed':'Mixed','motor':'Motor','mem':'Memory'}, inplace=True)
DS_violin=pd.melt(DS_folds, id_vars=['train','test'],value_vars=['Motor','Mixed','Memory'], var_name='task', value_name='acc')
DS_violin.to_csv(outDir+'acc/DS/folds/allFolds.csv', index=False)

#SScondensed 
for sub in subList:
    sub_tmp=pd.DataFrame()
    for train in taskList:
        for test in taskList:
            if train==test:
                continue
            tmp=pd.read_csv(outDir+'acc/SS/folds/'+train+test+sub+'.csv',usecols=[1,2,3])
            sub_tmp=pd.concat([sub_tmp,tmp])
    #return sub_tmp
    sub_tmp.to_csv(outDir+'acc/SS/folds/'+sub+'.csv', index=False)

#BScondensed 
for train_sub in subList:
    subF=pd.DataFrame()
    for test_sub in subList:
        if train_sub==test_sub:
            continue
        for train_task in taskList:
            for test_task in taskList:
                if train_task==test_task:
                    continue
                tmp=pd.read_csv(outDir+'acc/BS/folds/'+train_sub+test_sub+train_task+test_task+'.csv', usecols=[1,2,3,4,5])
                subF=pd.concat([subF, tmp])
    subF.to_csv(outDir+'acc/BS/folds/'+train_sub+'.csv', index=False)
#SS once all folds per tasks are condensed into subs
SS=pd.DataFrame()
for sub in subList:
    tmp=pd.read_csv(outDir+'acc/SS/folds/'+sub+'.csv')
    tmp=pd.melt(tmp, id_vars=['train','test'], value_vars=[sub], var_name='sub', value_name='acc')
    SS=pd.concat([SS,tmp])
SS.to_csv(outDir+'acc/SS/folds/allFolds.csv', index=False)
#BS once all folds are condensed into subs
BS=pd.DataFrame()
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
for sub in subList:
    tmp=pd.read_csv(outDir+'acc/BS/folds/'+sub+'.csv')
    BS=pd.concat([BS,tmp])
BS.to_csv(outDir+'acc/BS/folds/allFolds.csv', index=False)
#reformedFolds CV in results.py code

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
